In [37]:
import Random
using Distributions
using Statistics

In [3]:
include("Storage.jl")
include("GillespieFunctions.jl")
include("Rates.jl")

tt_protein_fb_rates! (generic function with 1 method)

In [4]:
function balance_test(initialstate::Vector{Int64},params::Vector{Float64},numsteps::Int64,maxsteps::Int64,rng::Random.AbstractRNG)
    statevector = initialstate
    outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
    calcstor_main = Balance_with_Derivative()
    calcstor = calcstor_main.balancedata
    endstate,steps,weightsum = direct_gillespie!(initialstate,params,outcomes,tt_protein_fb_rates!,calcstor,updatestorage!,rng,numsteps,maxsteps,50000)

    println("Flux Balance on x")
    println(relative_error(calcstor.meanRpX,calcstor.meanRmX))

    println("Flux Balance on y")
    println(relative_error(calcstor.meanRpY,calcstor.meanRmY))

    println("Cov Balance on x,x")
    println(cov_balance_comp(calcstor.CXRmX/weightsum,calcstor.CXRmX/weightsum,
                            calcstor.meanRpX+calcstor.meanRmX,calcstor.CXRpX/weightsum,calcstor.CXRpX/weightsum))
    println("Cov Balance on y,y")
    println(cov_balance_comp(calcstor.CYRmY/weightsum,calcstor.CYRmY/weightsum,
                            calcstor.meanRpY+calcstor.meanRmY,calcstor.CYRpY/weightsum,calcstor.CYRpY/weightsum))
    println("Cov Balance on y,x")
    println(cov_balance_comp(calcstor.CYRmX/weightsum,calcstor.CXRmY/weightsum,
                            0,calcstor.CYRpX/weightsum,calcstor.CXRpY/weightsum))
    println(steps)
    println(endstate)
    println(calcstor.meanx)
    println(calcstor.meany)


    return calcstor
end

balance_test (generic function with 1 method)

In [5]:
@time calcs = balance_test([10,50],[10.,1.,4.,1.,-1.,50.],10^2,10^10,Random.MersenneTwister())

Flux Balance on x
0.03699327873888824
Flux Balance on y
0.05206147913303602
Cov Balance on x,x
0.008525856319191604
Cov Balance on y,y
0.016751712367545402
Cov Balance on y,x
0.4303883762244462
50914
[6, 20]
6.696495802213428
28.21756412759429
  0.128658 seconds (387.05 k allocations: 21.241 MiB, 97.19% compilation time)


BalanceEqData(6.696495802213428, 28.21756412759429, 77.84016064639447, 742.3076660279346, 86.89127480438832, 6.447484376996817, 6.695160308490341, 26.780641233961365, 28.25145370131005, 76.80809408116403, -6.1247820090086185, 92.82591976522471, -57.46944903622024, 81.7860467747249, 307.2323763246561, 733.5989802623286, 371.30367906089884, -0.08395932510593487)

In [6]:
function data_collection(initialstate::Vector{Int64},params::Vector{Float64},numsteps::Int64,maxsteps::Int64,rng::Random.AbstractRNG)
    outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
    datastor = BalanceEqData()
    endstate,steps,weightsum = direct_gillespie!(initialstate,params,outcomes,tt_protein_fb_rates!,datastor,updatestorage!,rng,numsteps,maxsteps,50000)
    return datastor.meanx,datastor.meany,datastor.meanF_y,datastor
end


data_collection (generic function with 1 method)

In [11]:
x,y,f_y,d = data_collection([10,50],[10.,5.,10.,1.,-3.,5.],10^7,10^10,Random.MersenneTwister())

(0.8262001436856997, 6.7979795915029895, -0.9533316759193812, BalanceEqData(0.8262001436856997, 6.7979795915029895, 1.9308884332341724e6, 2.3421685270311877e7, 202521.91992837333, 3.515202800850065, 3.5132929166229556, 7.026585833245911, 7.026189553713489, 7.456448923047642e6, 873305.0140694319, 6.415969381453213e6, -1.6630788405519536e7, -795153.0935573414, 1.4912897846095284e7, 2.2672290708936676e7, 1.2831938762906427e7, -0.9533316759193812))

In [126]:
function CVratio(meanxs,meanys)
    return (std(meanys,corrected=false)/mean(meanys))/(std(meanxs,corrected=false)/mean(meanxs))
end
function pert_dot(initialparameters,numsamples,initialstate,numsteps,maxsteps)
    mean_xs = Vector{Float64}(undef,0)
    mean_ys = Vector{Float64}(undef,0)
    mean_fys = Vector{Float64}(undef,0)
    params = copy(initialparameters)
    outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
    
    for i in 1:numsamples
        rng = Random.MersenneTwister()
        params[i%4+1] *= rand(Uniform(0.9, 1.1))
        datastor = BalanceEqData()
        endstate,steps,weightsum = direct_gillespie!(initialstate,params,outcomes,tt_protein_fb_rates!,datastor,updatestorage!,rng,numsteps,maxsteps)
        append!(mean_xs,datastor.meanx)
        append!(mean_ys,datastor.meany)
        append!(mean_fys,datastor.meanF_y)
        println(params)
        params[i%4+1] = initialparameters[i%4+1]
        println(datastor.meanx)
        println(datastor.meany)
    end

    mean_dxs = mean_xs .- mean(mean_xs)
    mean_dys = mean_ys .- mean(mean_ys)

    return CVratio(mean_xs,mean_ys),cor(mean_xs,mean_ys),mean(mean_fys)
end

pert_dot (generic function with 1 method)

In [130]:
C,rho,avgFy = pert_dot([10.,1.,10.,1.,0.,10.],50,[10,10],10^6,10^8)
println(join((1/C,C,rho,avgFy),' '))

[10.0, 1.064585742623212, 10.0, 1.0, 0.0, 10.0]
4.7021230554975375
46.90280078199826
[10.0, 1.0, 9.304353960657691, 1.0, 0.0, 10.0]
5.016362355219791
46.55825335794532
[10.0, 1.0, 10.0, 1.0616641408374194, 0.0, 10.0]
5.012300413780164
47.11459623533006
[9.072912565877926, 1.0, 10.0, 1.0, 0.0, 10.0]
4.538411342387438
45.29860643454072
[10.0, 0.9897202364733025, 10.0, 1.0, 0.0, 10.0]
5.037650936742899
50.22915707271101
[10.0, 1.0, 10.608393284138577, 1.0, 0.0, 10.0]
5.010051833832056
53.025746708557335
[10.0, 1.0, 10.0, 1.0200628842534203, 0.0, 10.0]
5.013870454335425
49.01432877498337
[9.40958413232023, 1.0, 10.0, 1.0, 0.0, 10.0]
4.71854581879848
47.047377163633364
[10.0, 0.9030028798273947, 10.0, 1.0, 0.0, 10.0]
5.531644219777885
55.16874312492964
[10.0, 1.0, 10.463383779104028, 1.0, 0.0, 10.0]
5.015916792454711
52.309447682764
[10.0, 1.0, 10.0, 0.9335111637854254, 0.0, 10.0]
5.0120691817234375
53.607798214251375
[9.573974334328422, 1.0, 10.0, 1.0, 0.0, 10.0]
4.795176091897978
47.83873

0.013157894736842105

9.455873842491803